# AutoGen Tool Gebruik Voorbeeld


## Importeer de benodigde pakketten


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## De Client Maken

In dit voorbeeld gebruiken we [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) voor toegang tot de LLM.

Het `model` is gedefinieerd als `gpt-4o-mini`. Probeer het model te wijzigen naar een ander model dat beschikbaar is in de GitHub Models-marktplaats om de verschillende resultaten te zien.

Als snelle test voeren we gewoon een eenvoudige prompt uit - `Wat is de hoofdstad van Frankrijk`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## De functies definiëren

In dit voorbeeld geven we de agent toegang tot een hulpmiddel dat een functie is met een lijst van beschikbare vakantiebestemmingen en hun beschikbaarheid.

Je kunt je voorstellen dat dit een scenario is waarin een reisagent bijvoorbeeld toegang heeft tot een reisdatabase.

Terwijl je dit voorbeeld doorloopt, kun je gerust proberen nieuwe functies en hulpmiddelen te definiëren die de agent kan aanroepen.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## De functie-tool definiëren  
Om de agent `vacation_destinations` als een `FunctionTool` te laten gebruiken, moeten we deze als zodanig definiëren.  

We zullen ook een beschrijving geven van de tool, wat nuttig is voor de agent om te begrijpen waarvoor die tool wordt gebruikt in relatie tot de taak die de gebruiker heeft aangevraagd.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Het definiëren van de agent

Nu kunnen we de agent maken in de onderstaande code. We definiëren het `system_message` om de agent instructies te geven over hoe gebruikers te helpen bij het vinden van vakantiebestemmingen.

We stellen ook de parameter `reflect_on_tool_use` in op true. Dit stelt de LLM in staat om de respons van de toolaanroep te gebruiken en deze in natuurlijke taal te versturen.

Je kunt de parameter op false instellen om het verschil te zien.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## De Agent Uitvoeren

Nu kunnen we de agent uitvoeren met het eerste bericht van de gebruiker waarin wordt gevraagd om een reis naar Tokio te maken.

Je kunt deze stadsbestemming wijzigen om te zien hoe de agent reageert op de beschikbaarheid van de stad.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
